<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [ ]:
# librerias

import pandas as pd
import numpy as np


In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import pylab as plt

In [ ]:
# datos
Cat=pd.read_csv('data/cookie_cats.csv')
Cat.head()

In [ ]:
Cat.info()

In [ ]:
ret_1=pd.crosstab(Cat.version, Cat.retention_1)
ret_7=pd.crosstab(Cat.version, Cat.retention_7)

ret_1, ret_7

In [ ]:
rete_1=pd.crosstab(Cat.version, Cat.retention_1).plot(kind='bar');
rete_7=pd.crosstab(Cat.version, Cat.retention_7).plot(kind='bar');

In [ ]:
# transformacion
Cat.version=np.where(Cat.version=='gate_30', '30', '40')

Cat.retention_1=np.where(Cat.retention_1==True, 1, 0)
Cat.retention_7=np.where(Cat.retention_7==True, 1, 0)

Cat.head()

In [ ]:
# muestreo
Muestra_30=len(Cat[Cat.version=='30'])
Muestra_40=len(Cat[Cat.version=='40'])

Muestra_30, Muestra_40

In [ ]:
control = Cat[Cat.version=='30'].sample(n=44000, random_state=42)

tratamiento = Cat[Cat.version=='40'].sample(n=44000, random_state=42)

ab_test = pd.concat([control, tratamiento], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

In [ ]:
ab_test.tail()

In [ ]:
ab_test.version.value_counts()

In [ ]:
# testeo para retention_1


In [ ]:
control_res=ab_test[ab_test.version=='30']['retention_1']
tratamiento_res=ab_test[ab_test.version=='40']['retention_1']

n_con=control_res.count()       
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  


observaciones=[n_con, n_trat]   

z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones) 


(control_a, tratamiento_a), (control_b, tratamiento_b) = proportion_confint(conversiones,
                                                                            nobs=observaciones,

                                                                            alpha=0.05)
print(f'z-score: {z_score:.2f}')
print(f'p-valor: {p_valor:.3f}')
print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')
print(f'intervalo conf 95% para grupo tratamiento: [{tratamiento_a:.3f}, {tratamiento_b:.3f}]')


In [ ]:
ab_test.groupby('version').retention_1.mean()

In [ ]:
ab_test.groupby('version').retention_1.std()

In [ ]:
x=[i*.1 for i in range(-5, 8)]  

control=[norm.pdf(x, tasas_conversion.iloc[0][0], tasas_conversion.iloc[0][1]) for x in x] 

tratamiento=[norm.pdf(x, tasas_conversion.iloc[1][0], tasas_conversion.iloc[1][1]) for x in x] 

plt.plot(x, control)
plt.plot(x, tratamiento)
plt.legend(['control', 'tratamiento']);

In [ ]:
tasas_conversion=ab_test.groupby('version')['retention_1']


tasas_conversion=tasas_conversion.agg([np.mean, std, error_std])
tasas_conversion.columns=['conversion_rate', 'std_deviation', 'std_error']


tasas_conversion.style.format('{:.3f}')

x=[i*.1 for i in range(-5, 15)]  

control=[norm.pdf(x, tasas_conversion.iloc[0][0], tasas_conversion.iloc[0][1]) for x in x] 

tratamiento=[norm.pdf(x, tasas_conversion.iloc[1][0], tasas_conversion.iloc[1][1]) for x in x] 

plt.plot(x, control)
plt.plot(x, tratamiento)
plt.legend(['control', 'tratamiento']);

In [ ]:
# testeo para retention_7
control_res=ab_test[ab_test.version=='30']['retention_7']
tratamiento_res=ab_test[ab_test.version=='40']['retention_7']

n_con=control_res.count()       
n_trat=tratamiento_res.count()

conversiones=[control_res.sum(), tratamiento_res.sum()]  


observaciones=[n_con, n_trat]   

z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones) 


(control_a, tratamiento_a), (control_b, tratamiento_b) = proportion_confint(conversiones,
                                                                            nobs=observaciones,

                                                                            alpha=0.05)
print(f'z-score: {z_score:.2f}')
print(f'p-valor: {p_valor:.3f}')
print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')
print(f'intervalo conf 95% para grupo tratamiento: [{tratamiento_a:.3f}, {tratamiento_b:.3f}]')

In [ ]:
ab_test.groupby('version').retention_7.mean()

In [ ]:
ab_test.groupby('version').retention_7.std()

In [ ]:
tasas_conversion=ab_test.groupby('version')['retention_7']


tasas_conversion=tasas_conversion.agg([np.mean, std, error_std])
tasas_conversion.columns=['conversion_rate', 'std_deviation', 'std_error']


tasas_conversion.style.format('{:.3f}')

x=[i*.1 for i in range(-5, 15)]  

control=[norm.pdf(x, tasas_conversion.iloc[0][0], tasas_conversion.iloc[0][1]) for x in x] 

tratamiento=[norm.pdf(x, tasas_conversion.iloc[1][0], tasas_conversion.iloc[1][1]) for x in x] 

plt.plot(x, control)
plt.plot(x, tratamiento)
plt.legend(['control', 'tratamiento']);

In [ ]:
# conclusiones
Tras ver los datos obtenidos tanto como esta actualmente como cambiando el paywall (de puerta 30 a puerta 40),
he visto que no hay mucho cambio, apenas se nota  diferencia alguna entre el control y tratamiento,
en la unica que se nota que hay algo de diferencia es el control de la retención de 7 dias, el cambio de versión,
es decir el tratamiento esta un poco más por encima que el control, pero apenas se nota la diferencia.